# Загрузка картинок и модулей

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install sklearn
!pip  install livelossplot
!pip install tensorflow-addons
!pip install -U tensorflow
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Input, Dense, Dropout,MaxPooling2D , \
    Flatten, PReLU, BatchNormalization, Layer, Lambda, LeakyReLU, SpatialDropout2D, SeparableConv2D,GlobalAveragePooling2D 
from tensorflow.keras.models import Model
import tensorflow as tf 
from livelossplot.tf_keras import PlotLossesCallback
import os
import glob
import numpy as  np
import  pandas as pd
from PIL import Image
import shutil
import cv2
import difflib
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
!pip install segmentation_models
import segmentation_models as sm
import tensorflow_addons as tfa
from tqdm.auto import tqdm
from skimage.transform import resize
 

     |████████████████████████████████| 1.1 MB 7.0 MB/s 
     |████████████████████████████████| 50 kB 7.6 MB/s 
Segmentation Models: using `keras` framework.


In [ ]:
!cp -r /content/drive/MyDrive/Hand_Segmentation . 
!unzip   -q ./Hand_Segmentation/test.zip -d test
!unzip   -q ./Hand_Segmentation/train.zip -d train

# Настройка

In [ ]:
BACKBONE = 'resnet34'
TAG = "big_shape"
CHECKPOINT_FILEPATH = '/content/drive/MyDrive/segmentation_rucode4_'+BACKBONE+'_'+TAG+'.h5'
DATASET_TRAIN_PATH = './train/train/'
DATASET_TEST_PATH = './test/test/images' 
LOAD_PATH = CHECKPOINT_FILEPATH
LOAD_MODEL = True
#LERNING_RATE = 3e-5
BATSH_SIZE = 16
EPOCHS = 100
INPUT_WIDTH = 640 
INPUT_HEIGHT = 480
sm.set_framework('tf.keras')
preprocess_input = sm.get_preprocessing(BACKBONE)

# Загрузка модели и датасета 

In [ ]:
import glob
df_train = pd.DataFrame ( glob.glob( os.path.join(DATASET_TRAIN_PATH, '*/*/*.png') ), columns=['path'])
df_train['filename'] = df_train['path'].apply(lambda x : x.split('/')[-1])
df_train['type'] = df_train['path'].apply(lambda x : x.split('/')[-2])
df_train['Class'] = df_train['path'].apply(lambda x : x.split('/')[-3])
df_train = df_train[df_train['type'] == 'images']
df_train['path_mask'] = df_train.apply(lambda x: os.path.join(DATASET_TRAIN_PATH, x.Class , 'segmentation' , x.filename )  , axis=1)
df_train = df_train.drop(columns=['type'])
datasetTrain_df ,datasetValid_df= train_test_split(df_train, test_size=0.2, random_state=74  )

In [ ]:
if LOAD_MODEL :
  model = keras.models.load_model(LOAD_PATH, compile=False)
else:
  model = sm.Linknet(BACKBONE, encoder_weights='imagenet',classes=1, activation='sigmoid' ,input_shape=(INPUT_HEIGHT , INPUT_WIDTH, 3))  
model.summary() 

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 bn_data (BatchNormalization)   (None, None, None,   9           ['data[0][0]']                   
                                3)                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, None, None,   0          ['bn_data[0][0]']                
                                3)                                                          

In [ ]:
def preprocess_mask(path,H=INPUT_HEIGHT,W=INPUT_WIDTH):
  mask_raw = tf.io.read_file(path)
  mask_tensor = tf.image.decode_image(mask_raw ,expand_animations=False)
  mask_tensor = tf.image.resize(mask_tensor, (H, W))
  mask_tensor =(mask_tensor[...,0] + mask_tensor[...,1] + mask_tensor[...,2] )
  mask_tensor = tf.clip_by_value(mask_tensor, clip_value_min=0, clip_value_max=1)
  mask_tensor = tf.expand_dims(mask_tensor ,-1) 
  mask_tensor.set_shape([H, W, 1])
  return mask_tensor

def preprocess_image(path,H=INPUT_HEIGHT,W=INPUT_WIDTH):
  image_raw = tf.io.read_file(path)
  image_tensor = tf.image.decode_image(image_raw ,expand_animations=False,channels=3)
  image_tensor = image_tensor[:, :, ::-1]
  image_tensor = tf.image.resize(image_tensor, (H, W))
  image_tensor.set_shape([H, W, 3])
  return image_tensor
 
def augmentation_dataset(image, mask):
  if tf.random.uniform(()) > 0.5:
        # use original image to preserve high resolution
        image = tf.image.central_crop(image, 0.95)
        mask = tf.image.central_crop(mask, 0.95)
        # resize
        image = tf.image.resize(image, ( INPUT_HEIGHT  , INPUT_WIDTH))
        mask = tf.image.resize(mask, ( INPUT_HEIGHT  , INPUT_WIDTH))  
  if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)
  if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_up_down(image)
        mask = tf.image.flip_up_down(mask)
  # if tf.random.uniform(()) > 0.5:
  #       image = tf.image.rot90(image)
  #       mask = tf.image.rot90(mask)
  return image, mask

def tf_dataset(x, y, batch=8, augmentation=True):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=1000)
    if  augmentation:
      dataset = dataset.map(lambda x,y: augmentation_dataset(preprocess_image(x), preprocess_mask(y)))
    else:
      dataset = dataset.map(lambda x,y: (preprocess_image(x), preprocess_mask(y)))
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset  

In [ ]:
datasetTrain = tf_dataset(datasetTrain_df['path'], datasetTrain_df['path_mask'] ,BATSH_SIZE)
datasetValid = tf_dataset(datasetValid_df['path'], datasetValid_df['path_mask'] ,BATSH_SIZE, augmentation=False) 
for x, y in datasetTrain:
    x = x[0]  
    y = y[0]  
    
    x = x.numpy()
    y = y.numpy()
    print(x.max())
    cv2.imwrite("image.png", x )
    #y = np.squeeze(y, axis=-1)
    cv2.imwrite("mask.png", y*255)
    break

230.0


# Обучение модели

In [ ]:
model.compile(
    tfa.optimizers.RectifiedAdam(),
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=CHECKPOINT_FILEPATH,
    monitor='val_iou_score',
    mode='max', 
    save_best_only=True)

In [ ]:
model.fit(
   datasetTrain,
   batch_size=BATSH_SIZE,
   epochs=EPOCHS,
   validation_data=datasetValid,
   callbacks=[ PlotLossesCallback() ,model_checkpoint_callback]
) 

Epoch 1/100
   3/1041 [..............................] - ETA: 10:01 - loss: 0.0074 - iou_score: 0.9945

KeyboardInterrupt: ignored

# Предсказание

In [ ]:
model = keras.models.load_model(LOAD_PATH, compile=False)

In [ ]:
 def rle_encoding(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

In [ ]:
import glob
df_test = pd.DataFrame ( glob.glob( os.path.join(DATASET_TEST_PATH, '*.png') ), columns=['path'])
df_test['filenames'] = df_test['path'].apply(lambda x: x.split('/')[-1])
df_test['id'] = df_test['filenames'].apply(lambda x: x.split('.')[-2])
df_test['id'] = df_test['id'].astype(int)
df_test = df_test.sort_values(by=['id'])
 

In [ ]:
ans=[]
for id,row in tqdm(df_test.iterrows ()) :
  x = preprocess_image(row['path'])
  x = np.expand_dims(x, axis=0)
  y = model.predict(x)
  y=y[0]
  y = np.asarray((y >=0.5))
  #y = resize(y, (480,640))
  ans.append(rle_encoding(y)) 

0it [00:00, ?it/s]

In [ ]:
df_test['image']="dataset/test/images/"+df_test['filenames']
df_test['segmentation']=np.asarray(ans)
df_test.index = list(df_test["id"])
ans_df = df_test.drop(columns=['path',"filenames","id"])
ans_df.to_csv("ans.csv",index=False )

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
